<h1> Databook that contains all the data scraped from websites, json endpoints, and official Riot API</h1>

<h2> Webscrape </h2>

<h4>Webscraping for list of all availible champions, respective winrates, pick rates, and ban rates. </h4>

In [34]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/')
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.endswith('.json') and not href =='-1.json':
        links.append(f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/{href}")

def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict,list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

champion_names = []
champion_ids = []

for url in links:
    champion_json = requests.get(url).json()
    
    id = json_extract(champion_json, 'id')
    name = json_extract(champion_json, 'name')
    champion_names.append(name[0])
    champion_ids.append(id[0])

championId_dict = dict(map(lambda i, j: (int(i), j), champion_ids, champion_names))

In [35]:
championId_dict #list of all champion names scraped from community dragon

{1: 'Annie',
 10: 'Kayle',
 101: 'Xerath',
 102: 'Shyvana',
 103: 'Ahri',
 104: 'Graves',
 105: 'Fizz',
 106: 'Volibear',
 107: 'Rengar',
 11: 'Master Yi',
 110: 'Varus',
 111: 'Nautilus',
 112: 'Viktor',
 113: 'Sejuani',
 114: 'Fiora',
 115: 'Ziggs',
 117: 'Lulu',
 119: 'Draven',
 12: 'Alistar',
 120: 'Hecarim',
 121: "Kha'Zix",
 122: 'Darius',
 126: 'Jayce',
 127: 'Lissandra',
 13: 'Ryze',
 131: 'Diana',
 133: 'Quinn',
 134: 'Syndra',
 136: 'Aurelion Sol',
 14: 'Sion',
 141: 'Kayn',
 142: 'Zoe',
 143: 'Zyra',
 145: "Kai'Sa",
 147: 'Seraphine',
 15: 'Sivir',
 150: 'Gnar',
 154: 'Zac',
 157: 'Yasuo',
 16: 'Soraka',
 161: "Vel'Koz",
 163: 'Taliyah',
 164: 'Camille',
 166: 'Akshan',
 17: 'Teemo',
 18: 'Tristana',
 19: 'Warwick',
 2: 'Olaf',
 20: 'Nunu & Willump',
 200: "Bel'Veth",
 201: 'Braum',
 202: 'Jhin',
 203: 'Kindred',
 21: 'Miss Fortune',
 22: 'Ashe',
 221: 'Zeri',
 222: 'Jinx',
 223: 'Tahm Kench',
 23: 'Tryndamere',
 233: 'Briar',
 234: 'Viego',
 235: 'Senna',
 236: 'Lucian',
 2

In [ ]:
import json
with open('championId_dict.json', 'w') as f:
        json.dump(championId_dict, f)

In [5]:
import os
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
base_dir = os.path.abspath(os.path.dirname('routes'))

championId_path = os.path.join(base_dir, 'routes/data/championId_dict.json')
csv_path = os.path.join(base_dir, 'routes/data/champion_data_df.csv')

champion_data_df = pd.read_csv(csv_path)
champion_data_df = champion_data_df.drop(columns=['Unnamed: 0'])

with open(championId_path, 'r') as file:
    championId_dict = json.load(file)

In [6]:
import re
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get('https://u.gg/lol/champions', headers=headers)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')
baseURL = "https://u.gg/"

champContainer = soup.find('div', class_='champion-home-page')
champion_data = []
if champContainer:
    links = champContainer.find_all('a')

    for link in links:
        url = baseURL + link['href']
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            html = resp.text
            soup1 = BeautifulSoup(html, 'html.parser')
            rankingStats = soup1.find('div', class_='champion-ranking-stats-normal')
            if rankingStats:
                text = rankingStats.text.strip()
                tier_pattern = r"([A-S]\+?Tier)"
                redTier_pattern = r"([A-S]\+)"
                win_rate_pattern = r"(\d+(\.\d+)?%)Win Rate"
                rank_pattern = r"(\d+ / \d+)Rank"
                pick_rate_pattern = r"(\d+\.\d+%)Pick Rate"
                ban_rate_pattern = r"(\d+\.\d+%)Ban Rate"
                matches_pattern = r"(\d+,?\d*,?\d*)Matches"
                
                # Apply patterns
                tier = re.search(tier_pattern, text).group(1)
                win_rate = re.search(win_rate_pattern, text).group(1)
                rank = re.search(rank_pattern, text).group(1)
                pick_rate = re.search(pick_rate_pattern, text).group(1)
                ban_rate = re.search(ban_rate_pattern, text).group(1)
                matches = re.search(matches_pattern, text).group(1)

                numbers = rank.split('/')

                # Strip whitespace and convert to integers
                numerator = int(numbers[0].strip())
                denominator = int(numbers[1].strip())

                # Perform division
                rank = f"{((numerator / denominator)*100):.2f}%"

                name = soup1.find('span', class_='champion-name').text.strip()
                if name == 'Red Kayn':
                    name = 'Kayn'

                champion_data.append({
                'name' : name,
                'tier': tier,
                'win_rate': win_rate,
                'rank': rank,
                'pick_rate': pick_rate,
                'ban_rate': ban_rate,
                'matches': matches
                })

In [8]:
champion_data_df = pd.DataFrame(champion_data)
champion_data_df #Data of all champions scraped from u.gg

,name,tier,win_rate,rank,pick_rate,ban_rate,matches
0,Aatrox,S+Tier,50.27%,52.46%,12.5%,16.5%,"124,992"
1,Ahri,STier,50.25%,52.54%,7.7%,5.6%,"77,346"
2,Akali,CTier,49.46%,77.97%,4.6%,10.4%,"46,341"
3,Akshan,S+Tier,52.52%,5.08%,2.5%,5.9%,"25,387"
4,Alistar,ATier,50.25%,46.67%,5.8%,2.1%,"57,730"
...,...,...,...,...,...,...,...
163,Zeri,BTier,49.86%,50.00%,10.7%,5.3%,"107,328"
164,Ziggs,CTier,48.35%,88.14%,1.6%,0.4%,"16,311"
165,Zilean,ATier,50.96%,26.67%,2.2%,0.6%,"22,210"
166,Zoe,ATier,50.44%,47.46%,2.2%,1.0%,"22,207"


In [9]:
champion_data_df.to_csv('champion_data_df.csv')

<h2> Database Postgresql </h2>

<h4> Connection to DataBase for easy retrieval/insertion of data to prevent exceeding ratelimit on API and general efficient performance </h4>

In [5]:
from sqlalchemy import text, create_engine

db_username = "postgres"
db_password = "123123123"
db_host = "localhost"
db_port = "5432"
db_name = "lol_analytics"

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
        connection_url = 'postgresql+psycopg2://'+db_username+':'+db_password+'@'+db_host+':'+db_port+'/'+db_name
        return connection_url

conn_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine = create_engine(conn_url, pool_recycle=3600)

connection = db_engine.connect()

In [8]:
with db_engine.connect() as connection:
    leaderboard_df = pd.read_sql(text('SELECT * FROM "Top50SoloQ".leaderboard;'), connection)

leaderboard_df #List of top 300 players in LOL

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14,2154,I,173,100,True,False,False,True
1,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8,1661,I,199,137,True,False,False,False
2,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA,1551,I,188,136,True,False,False,True
3,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U,1481,I,140,86,True,False,False,False
4,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...,1475,I,235,189,True,False,False,False
...,...,...,...,...,...,...,...,...,...
6302,p9SSjKDI7ZVLkjuBPRKJZkrPAn7bs97FW2YPpAgQshwjBM...,0,I,25,21,False,False,False,False
6303,6LAr20zjLNQzmKNPJzKnlx0Ry-o5argtvoLUITEBeDtdsoA,0,I,39,35,False,False,False,False
6304,1yoQdWF52dhGvdn8eDdYx6Rg_SFN56wkpiPF8TmGSgsBdl...,0,I,27,22,False,False,False,False
6305,_T4EJRjPENc66ypQpv5TUoT7cer5OQi9OdGyWHVwL_ot0oU,0,I,39,25,False,False,False,False


In [6]:
with db_engine.connect() as connection:
    playerId_df = pd.read_sql(text('SELECT * FROM "playerId".summonerdata;'), connection)
playerId_df #list of top 90 players in LOL and their ids

,index,puuid,summonerId
0,0,Zl5uf2ERyvcZJD_4ELxzgc9SCaltAuDOjAIiuj2sdknOA1...,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14
1,1,lWMqB3Y5AuREUMTIq2uNUzYosZFcrsGm3UwKjuW8ZdQxQS...,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8
2,2,5XqBXMuSPFUrpk807hIZ5jJ-IGj2uyOgySWq6-GrXlDcmM...,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA
3,3,iOI9wt8myzTiOUZpdqYumAQo1490Axk4sFvKp1ZTtz0fui...,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U
4,4,_7r_DQhl9ENq3mc9XYlpuESPeRyv3CS2lLLHrSN-WkrDvd...,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...
...,...,...,...
6302,6302,DQxONVAoyDvcqNCbMR3j3ROB9qN_famSLFynNLIXVLr7Ht...,p9SSjKDI7ZVLkjuBPRKJZkrPAn7bs97FW2YPpAgQshwjBM...
6303,6303,XElf8T5P2XMu-sKuG6E0h2m7rhsZW1mkLZpKi1oSkcSAar...,6LAr20zjLNQzmKNPJzKnlx0Ry-o5argtvoLUITEBeDtdsoA
6304,6304,nCfo4k3xqrPJfG-uYKPe0J6xjwtlKwWRDvjy6LyjFBZim9...,1yoQdWF52dhGvdn8eDdYx6Rg_SFN56wkpiPF8TmGSgsBdl...
6305,6305,pnuAsi-arvwFFD7bGf6CuF3WuE3nD9IvlnzmCXb7JliRtG...,_T4EJRjPENc66ypQpv5TUoT7cer5OQi9OdGyWHVwL_ot0oU


In [9]:
shuffled_df = leaderboard_df.sample(frac=1).reset_index(drop=True)

In [37]:
shuffled_df

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,9iHq8HRSmCl2NAk0k5YECg3yTSncshXVf3yMhl9F69Rm2QM,500,I,82,64,False,False,True,False
1,A0Af8n_gR8YOJ751X8SZdpbWrzBbIqk3b-LJ6JtNaWcr-d...,68,I,85,73,False,False,False,False
2,mkCHwytKamMbukW7wmT2jLnE3qSaSyBM7AHgV3t7SGiGgbE,59,I,254,266,False,False,True,False
3,GRf_IoNvPsSXYQpFNqYaFF1p344rL0ZIO3-7zq1-a9b0iy...,172,I,50,39,False,False,False,False
4,W031no0za7A5Nx9Khkz8mHToUBJy4XnRBnS_UKBSHaV51h...,214,I,35,19,False,False,False,True
...,...,...,...,...,...,...,...,...,...
6302,i3SeFNXb7nBoEDSYfJwZu68bwDsqpsm5NaVXtJzjGqFkhhw,357,I,119,101,False,False,False,False
6303,p9isuwtqtvu3V6Wk3cGvQiGVaR9d5kcbkWoUIoU0hMpgMAM,63,I,21,13,False,False,True,False
6304,vqXteg7UyDD2RTd0Ut8vzYLgd-sCZEaVrp9DijpqgKkuUE0,172,I,74,72,False,False,False,False
6305,R8Ue6skTtovNVFtEG7NtIGrB4N2ELzxtlJC3cVrav7Kzt-I,61,I,36,33,False,False,False,False


<h2> Data Collection </h2>

<h4> API functions </h4>

In [27]:
api_key = "RGAPI-c8d92844-a41a-49d1-ab74-5891fd2bcb16"

def get_puuid_fromSummonerId(summonerId):
    puuid = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}").json()['puuid']
    return puuid

def getLeaderBoard(league):
    get_top50_url = f"https://na1.api.riotgames.com/lol/league/v4/{league}leagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(get_top50_url).json()

def get_matches(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=25&api_key={api_key}"
    return requests.get(url).json()

def get_matchData(matchId):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    return requests.get(url).json()

<h4> Helper Functions </h4>

In [46]:
import time

def TopPlayers (): #function to get all players in the top 3 LOL leagues
    chal = getLeaderBoard('challenger')
    gm = getLeaderBoard('grandmaster')
    master = getLeaderBoard('master')

    chal_df = pd.DataFrame(chal['entries'])
    gm_df = pd.DataFrame(gm['entries'])
    master_df = pd.DataFrame(master['entries'])

    lb_df = pd.concat([chal_df, gm_df, master_df], ignore_index=True).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    lb_df = lb_df.set_index('summonerId')

    lb_df.to_sql('leaderboard', con=connection, schema='Top50SoloQ', if_exists='replace')

def PuuidWSummonerId(): #function to convert summonerId to Puuid and store it for easy conversions
    puuid_list = []
    count = 0
    for player in leaderboard_df['summonerId']:
        if count == 98:
            count = 0
            time.sleep(90) #wait for ratelimit on api
        
        puuid_list.append({'puuid' : get_puuid_fromSummonerId(player),
                        'summonerId': player})
        count += 1

    puuid_df = pd.DataFrame(puuid_list)
    puuid_df.to_sql(name='summonerdata', con=connection, schema='playerId', if_exists='replace', index=True)

def processMatchInfoV2(match):
    redTeamChamps = []
    redTeamRoles = []
    redTeamWRs = []
    redTeamPickRates = []
    redTeamBanRates = []
    redTeamTiers = []
    redTeamRanks = []

    blueTeamChamps = []
    blueTeamRoles = []
    blueTeamWRs = []
    blueTeamPickRates = []
    blueTeamBanRates = []
    blueTeamTiers = []
    blueTeamRanks = []

    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0
    #0 indicates blue side win, 1 indicates red side win

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    def get_championData(name):
        temp_df = champion_data_df[champion_data_df['name'].str.contains(name, case=False)]
        wr = temp_df['win_rate'].str.rstrip('%').astype(float).iloc[0]
        pr = temp_df['pick_rate'].str.rstrip('%').astype(float).iloc[0]
        br = temp_df['ban_rate'].str.rstrip('%').astype(float).iloc[0]
        tier = temp_df['tier'].iloc[0]
        rank = temp_df['rank'].str.rstrip('%').astype(float).iloc[0]

        return wr, pr, br, tier, rank
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            if player['championId'] in championId_dict:
                wr, pr, br, tier, rank = get_championData(championId_dict[player['championId']])
                redTeamChamps.append(championId_dict[player['championId']])
            redTeamRoles.append(player['teamPosition'])
            redTeamWRs.append(wr)
            redTeamPickRates.append(pr)
            redTeamBanRates.append(br)
            redTeamTiers.append(tier)
            redTeamRanks.append(rank)
        else:
            if player['championId'] in championId_dict:
                wr, pr, br, tier, rank = get_championData(championId_dict[player['championId']])
                blueTeamChamps.append(championId_dict[player['championId']])
            blueTeamRoles.append(player['teamPosition'])
            blueTeamWRs.append(wr)
            blueTeamPickRates.append(pr)
            blueTeamBanRates.append(br)
            blueTeamTiers.append(tier)
            blueTeamRanks.append(rank)
    
    match_info = pd.DataFrame([{
        'outcome' : outcome,

        #red team
        'redTeam' : redTeamChamps,
        'redTeamRoles' : redTeamRoles,
        'redTeamWinRates' : redTeamWRs,
        'redTeamPickRatesm' : redTeamPickRates,
        'redTeamBanRates' : redTeamBanRates,
        'redTeamTiers' : redTeamTiers,
        'redTeamRanks' : redTeamRanks,
        #blue team
        'blueTeam' : blueTeamChamps,
        'blueTeamRoles' : blueTeamRoles,
        'blueTeamWinRates' : blueTeamWRs,
        'blueTeamPickRatesm' : blueTeamPickRates,
        'blueTeamBanRates' : blueTeamBanRates,
        'blueTeamTiers' : blueTeamTiers,
        'blueTeamRanks' : blueTeamRanks,
    }])
    return match_info

In [45]:
def processMatchInfoV3(match): #Cleans up individual Match data for dataframe that feeds to AI
    redTeamChamps = []
    redTeamRoles = []
    blueTeamChamps = []
    blueTeamRoles = []
    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    # 2 indicates Red win, 0 indicates Blue win
    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            redTeamChamps.append(player['championId'])
            redTeamRoles.append(player['teamPosition'])
        else:
            blueTeamChamps.append(player['championId'])
            blueTeamRoles.append(player['teamPosition'])
    
    matchInfo = pd.DataFrame()

    for i in range(0,5):
        match = pd.DataFrame([{
            'outcome' : outcome,

            'redTeam' : redTeamChamps[i],
            'blueTeam' : blueTeamChamps[i],

            'Role' : redTeamRoles[i]
        }])
        matchInfo = pd.concat([matchInfo, match])

    return matchInfo

In [43]:
def processMatchInfoV1(match): #FIND THE MATCHUP WINRATE
    redTeamChamps = []
    redTeamRoles = []
    blueTeamChamps = []
    blueTeamRoles = []
    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            redTeamChamps.append(player['championId'])
            redTeamRoles.append(player['teamPosition'])
        else:
            blueTeamChamps.append(player['championId'])
            blueTeamRoles.append(player['teamPosition'])
    
    match_info = pd.DataFrame([{
        'outcome' : outcome,

        #red team
        'redTeam1' : redTeamChamps[0],
        'redTeam1R' : redTeamRoles[0],

        'redTeam2' : redTeamChamps[1],
        'redTeam2R' : redTeamRoles[1],

        'redTeam3' : redTeamChamps[2],
        'redTeam3R' : redTeamRoles[2],

        'redTeam4' : redTeamChamps[3],
        'redTeam4R' : redTeamRoles[3],

        'redTeam5' : redTeamChamps[4],
        'redTeam5R' : redTeamRoles[4],

        #blue team
        'blueTeam1' : blueTeamChamps[0],
        'blueTeam1R' : blueTeamRoles[0],

        'blueTeam2' : blueTeamChamps[1],
        'blueTeam2R' : blueTeamRoles[1],

        'blueTeam3' : blueTeamChamps[2],
        'blueTeam3R' : blueTeamRoles[2],

        'blueTeam4' : blueTeamChamps[3],
        'blueTeam4R' : blueTeamRoles[3],

        'blueTeam5' : blueTeamChamps[4],
        'blueTeam5R' : blueTeamRoles[4],
    }])
    return match_info

def merge_champion_statsV1(all_match_df, champion_data_df):
    all_matchData_df = all_match_df.copy()

    pattern = re.compile(r'^(redTeam\d+|blueTeam\d+)$')
    for index, row in all_match_df.iterrows():
        for col in all_match_df.columns:
            if pattern.match(col):
                champion_name = all_match_df.loc[index, col]
                stats = champion_data_df[champion_data_df['name'] == champion_name]
                if not stats.empty:
                    for stat_col in champion_data_df.columns:
                        if stat_col != 'name' and stat_col != 'matches' :
                            all_matchData_df.loc[index, f"{col}_{stat_col}"] = stats[stat_col].values[0]

    for count in range(1, 6):
        all_matchData_df[f'redTeam{count}_win_rate'] = all_matchData_df[f'redTeam{count}_win_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_pick_rate'] = all_matchData_df[f'redTeam{count}_pick_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_ban_rate'] = all_matchData_df[f'redTeam{count}_ban_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_rank'] = all_matchData_df[f'redTeam{count}_rank'].str.rstrip('%').astype(float)

        all_matchData_df[f'blueTeam{count}_win_rate'] = all_matchData_df[f'blueTeam{count}_win_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_pick_rate'] = all_matchData_df[f'blueTeam{count}_pick_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_ban_rate'] = all_matchData_df[f'blueTeam{count}_ban_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_rank'] = all_matchData_df[f'blueTeam{count}_rank'].str.rstrip('%').astype(float)

    return all_matchData_df

In [35]:
PuuidWSummonerId()

In [20]:
player_df = playerId_df.drop(columns='index')
player_df

,puuid,summonerId
0,Zl5uf2ERyvcZJD_4ELxzgc9SCaltAuDOjAIiuj2sdknOA1...,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14
1,lWMqB3Y5AuREUMTIq2uNUzYosZFcrsGm3UwKjuW8ZdQxQS...,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8
2,5XqBXMuSPFUrpk807hIZ5jJ-IGj2uyOgySWq6-GrXlDcmM...,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA
3,iOI9wt8myzTiOUZpdqYumAQo1490Axk4sFvKp1ZTtz0fui...,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U
4,_7r_DQhl9ENq3mc9XYlpuESPeRyv3CS2lLLHrSN-WkrDvd...,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...
...,...,...
6302,DQxONVAoyDvcqNCbMR3j3ROB9qN_famSLFynNLIXVLr7Ht...,p9SSjKDI7ZVLkjuBPRKJZkrPAn7bs97FW2YPpAgQshwjBM...
6303,XElf8T5P2XMu-sKuG6E0h2m7rhsZW1mkLZpKi1oSkcSAar...,6LAr20zjLNQzmKNPJzKnlx0Ry-o5argtvoLUITEBeDtdsoA
6304,nCfo4k3xqrPJfG-uYKPe0J6xjwtlKwWRDvjy6LyjFBZim9...,1yoQdWF52dhGvdn8eDdYx6Rg_SFN56wkpiPF8TmGSgsBdl...
6305,pnuAsi-arvwFFD7bGf6CuF3WuE3nD9IvlnzmCXb7JliRtG...,_T4EJRjPENc66ypQpv5TUoT7cer5OQi9OdGyWHVwL_ot0oU


<h4> Collection of list of matches played by top players </h4>

In [30]:
import time
uniqueMatches = []
count = 0
for player in player_df['puuid']:
    if count == 99:
        count = 0
        time.sleep(90)
        
    for match in get_matches(player):
        if match not in uniqueMatches:
            uniqueMatches.append(match)
    
    print(len(uniqueMatches))
    count += 1

print(len(uniqueMatches))

25
50
75
98
118
140
163
184
208
224
249
270
282
300
316
341
351
370
390
408
427
441
452
476
484
508
517
536
546
554
564
575
585
590
604
628
646
658
671
689
703
720
729
752
773
782
798
813
823
834
851
872
879
901
915
929
938
945
960
969
978
1000
1019
1030
1042
1060
1072
1085
1106
1127
1142
1164
1173
1192
1206
1226
1234
1248
1255
1274
1280
1292
1303
1314
1337
1352
1369
1388
1398
1408
1418
1424
1431
1448
1452
1466
1471
1477
1486
1493
1501
1503
1513
1525
1546
1555
1562
1573
1588
1599
1609
1621
1629
1633
1640
1648
1659
1672
1683
1701
1709
1717
1729
1741
1757
1773
1785
1794
1806
1822
1828
1832
1838
1854
1871
1876
1886
1902
1912
1923
1927
1941
1945
1953
1962
1979
1988
1994
2005
2018
2033
2043
2047
2063
2077
2092
2108
2122
2136
2144
2159
2167
2175
2195
2206
2218
2237
2259
2265
2284
2302
2321
2324
2334
2348
2360
2372
2381
2399
2410
2420
2433
2443
2457
2474
2481
2504
2515
2528
2537
2548
2560
2570
2582
2603
2614
2627
2638
2642
2647
2657
2669
2678
2680
2693
2700
2703
2715
2732
2735
2747
2757
2768


<h4> Then we proceed to look at each match individually and form 3 data tables for each type of AI model </h4>

In [48]:
dfv1 = pd.DataFrame()
dfv2 = pd.DataFrame()
dfv3 = pd.DataFrame()

matchesv1 = []
matchesv2 = []
matchesv3 = []

count = 0
for match in uniqueMatches:
    if count == 99:
        count = 0
        time.sleep(90)
    
    matchData = get_matchData(match)
    matchesv1.append(processMatchInfoV1(matchData))
    matchesv2.append(processMatchInfoV2(matchData))
    matchesv3.append(processMatchInfoV3(matchData))
    count+= 1

dfv1 = pd.concat(matchesv1)
dfv2 = pd.concat(matchesv2).reset_index(drop=True)
dfv3 = pd.concat(matchesv3)

IndexError: list index out of range

In [59]:
dfv1 = pd.concat(matchesv1)
dfv1

,outcome,redTeam1,redTeam1R,redTeam2,redTeam2R,redTeam3,redTeam3R,redTeam4,redTeam4R,redTeam5,...,blueTeam1,blueTeam1R,blueTeam2,blueTeam2R,blueTeam3,blueTeam3R,blueTeam4,blueTeam4R,blueTeam5,blueTeam5R
0,1,85,TOP,107,JUNGLE,238,MIDDLE,145,BOTTOM,497,...,77,TOP,76,JUNGLE,3,MIDDLE,81,BOTTOM,147,UTILITY
0,1,164,TOP,9,JUNGLE,42,MIDDLE,145,BOTTOM,111,...,19,TOP,30,JUNGLE,86,MIDDLE,81,BOTTOM,40,UTILITY
0,1,4,TOP,360,BOTTOM,81,MIDDLE,30,JUNGLE,111,...,221,TOP,121,JUNGLE,9,MIDDLE,145,BOTTOM,12,UTILITY
0,0,68,TOP,234,JUNGLE,910,MIDDLE,360,BOTTOM,111,...,77,TOP,63,JUNGLE,42,MIDDLE,222,BOTTOM,201,UTILITY
0,1,164,TOP,20,JUNGLE,4,MIDDLE,81,BOTTOM,26,...,236,TOP,245,JUNGLE,777,MIDDLE,21,BOTTOM,902,UTILITY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,24,TOP,154,JUNGLE,84,MIDDLE,145,BOTTOM,37,...,897,TOP,131,JUNGLE,18,MIDDLE,222,BOTTOM,53,UTILITY
0,1,72,TOP,154,JUNGLE,517,MIDDLE,119,BOTTOM,412,...,875,TOP,102,JUNGLE,101,MIDDLE,202,BOTTOM,3,UTILITY
0,1,516,TOP,9,JUNGLE,18,MIDDLE,221,BOTTOM,412,...,777,TOP,427,JUNGLE,101,MIDDLE,145,BOTTOM,57,UTILITY
0,1,85,TOP,63,JUNGLE,157,MIDDLE,222,BOTTOM,43,...,268,MIDDLE,234,JUNGLE,84,TOP,523,BOTTOM,37,UTILITY


In [57]:
dfv2 = pd.concat(matchesv2).reset_index(drop=True)
dfv2

,outcome,redTeam,redTeamRoles,redTeamWinRates,redTeamPickRatesm,redTeamBanRates,redTeamTiers,redTeamRanks,blueTeam,blueTeamRoles,blueTeamWinRates,blueTeamPickRatesm,blueTeamBanRates,blueTeamTiers,blueTeamRanks
0,1,"[Kennen, Rengar, Zed, Kai'Sa, Rakan]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[50.69, 48.61, 48.8, 50.8, 50.11]","[2.4, 2.7, 7.0, 33.7, 5.0]","[1.6, 5.4, 18.7, 12.4, 0.8]","[ATier, DTier, DTier, S+Tier, ATier]","[38.98, 88.24, 87.93, 29.63, 45.45]","[Udyr, Nidalee, Galio, Ezreal, Seraphine]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.88, 50.17, 52.42, 49.56, 50.63]","[3.4, 9.2, 3.8, 25.2, 4.7]","[2.3, 12.7, 1.5, 15.1, 1.1]","[STier, S+Tier, S+Tier, DTier, ATier]","[5.88, 50.98, 8.62, 74.07, 34.09]"
1,1,"[Camille, Fiddlesticks, Corki, Kai'Sa, Nautilus]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.58, 52.0, 47.91, 50.8, 49.37]","[6.3, 3.2, 5.9, 33.7, 13.1]","[7.2, 2.7, 4.2, 12.4, 20.7]","[S+Tier, STier, DTier, S+Tier, DTier]","[8.47, 1.96, 91.38, 29.63, 72.73]","[Warwick, Karthus, Garen, Ezreal, Janna]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.24, 49.06, 50.45, 49.56, 51.7]","[2.7, 3.4, 6.8, 25.2, 5.3]","[1.2, 13.1, 4.7, 15.1, 3.1]","[ATier, DTier, STier, DTier, S+Tier]","[13.73, 76.47, 45.76, 74.07, 9.09]"
2,1,"[Twisted Fate, Samira, Ezreal, Karthus, Nautilus]","[TOP, BOTTOM, MIDDLE, JUNGLE, UTILITY]","[49.93, 48.69, 49.56, 49.06, 49.37]","[3.4, 4.9, 25.2, 3.4, 13.1]","[3.7, 10.7, 15.1, 13.1, 20.7]","[BTier, DTier, DTier, DTier, DTier]","[58.62, 77.78, 74.07, 76.47, 72.73]","[Zeri, Kha'Zix, Fiddlesticks, Kai'Sa, Alistar]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.72, 50.38, 52.0, 50.8, 50.75]","[12.0, 10.1, 3.2, 33.7, 6.3]","[4.6, 9.9, 2.7, 12.4, 2.3]","[CTier, S+Tier, STier, S+Tier, STier]","[59.26, 47.06, 1.96, 29.63, 27.27]"
3,0,"[Rumble, Viego, Hwei, Samira, Nautilus]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.64, 49.82, 48.81, 48.69, 49.37]","[3.7, 11.5, 7.7, 4.9, 13.1]","[7.2, 6.7, 17.1, 10.7, 20.7]","[BTier, BTier, DTier, DTier, DTier]","[72.88, 66.67, 86.21, 77.78, 72.73]","[Udyr, Brand, Corki, Jinx, Braum]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.88, 47.99, 47.91, 51.36, 51.47]","[3.4, 2.7, 5.9, 14.9, 6.5]","[2.3, 15.2, 4.2, 5.9, 3.8]","[STier, DTier, DTier, S+Tier, S+Tier]","[5.88, 88.64, 91.38, 22.22, 13.64]"
4,1,"[Camille, Nunu & Willump, Twisted Fate, Ezreal...","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.58, 50.62, 49.93, 49.56, 50.25]","[6.3, 2.4, 3.4, 25.2, 2.3]","[7.2, 0.4, 3.7, 15.1, 0.7]","[S+Tier, ATier, BTier, DTier, ATier]","[8.47, 37.25, 58.62, 74.07, 40.91]","[Lucian, Ekko, Yone, Miss Fortune, Milio]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.56, 50.48, 49.42, 50.5, 50.03]","[9.8, 4.1, 6.4, 8.9, 7.6]","[3.2, 1.4, 6.7, 2.6, 5.0]","[CTier, ATier, DTier, STier, STier]","[70.37, 43.14, 74.14, 33.33, 50.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7594,1,"[Jax, Zac, Akali, Kai'Sa, Sona]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[50.1, 49.88, 49.0, 50.8, 51.2]","[6.4, 3.9, 4.8, 33.7, 2.8]","[13.1, 3.9, 12.0, 12.4, 0.2]","[S+Tier, BTier, DTier, S+Tier, ATier]","[61.02, 62.75, 82.76, 29.63, 20.45]","[K'Sante, Diana, Tristana, Jinx, Blitzcrank]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[46.62, 49.92, 50.73, 51.36, 49.8]","[4.3, 5.3, 11.2, 14.9, 6.6]","[2.0, 2.6, 19.0, 5.9, 25.2]","[DTier, BTier, S+Tier, S+Tier, ATier]","[94.92, 58.82, 37.93, 22.22, 61.36]"
7595,1,"[Skarner, Zac, Sylas, Draven, Thresh]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[45.82, 49.88, 50.83, 50.09, 50.04]","[1.8, 3.9, 7.4, 5.7, 12.6]","[18.4, 3.9, 9.3, 25.6, 4.8]","[DTier, BTier, S+Tier, S+Tier, STier]","[98.31, 62.75, 34.48, 40.74, 47.73]","[Sett, Shyvana, Xerath, Jhin, Galio]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.09, 51.05, 49.19, 50.05, 52.42]","[6.9, 3.3, 3.9, 15.2, 3.8]","[2.9, 0.9, 5.8, 3.4, 1.5]","[S+Tier, ATier, CTier, STier, S+Tier]","[16.95, 17.65, 77.27, 44.44, 8.62]"
7596,1,"[Ornn, Fiddlesticks, Tristana, Zeri, Thresh]","[TOP, JUNGLE, MIDDLE, BOTTOM, U

In [58]:
dfv3 = pd.concat(matchesv3)
dfv3

,outcome,redTeam,blueTeam,Role
0,1,85,77,TOP
0,1,107,76,JUNGLE
0,1,238,3,MIDDLE
0,1,145,81,BOTTOM
0,1,497,147,UTILITY
...,...,...,...,...
0,0,875,266,TOP
0,0,19,30,JUNGLE
0,0,103,18,MIDDLE
0,0,222,22,BOTTOM


In [60]:
dfv1.to_csv('dfv1.csv') #all 10 players individual
dfv2.to_csv('dfv2.csv') #all listed
dfv3.to_csv('dfv3.csv') #Matchup only